In [2]:
from paxml import main
import sys
import jax
import numpy as np


sys.path.append('/home/lishengping/projects/paxml/paxml')

from praxis import base_hyperparams
from praxis import base_input
from praxis import pax_fiddle
from praxis import py_utils
from paxml import checkpoints  # mapped to internal
from paxml import checkpoint_managers
from paxml import partitioning
from etils import epath
from paxml.checkpoint_metadata import make_metadata
import jax.numpy as jnp
import copy
from flax.traverse_util import flatten_dict, unflatten_dict, empty_node
from optax import MaskedNode
import jax
from praxis import optimizers
from praxis import schedules
import orbax
from paxml import main
from praxis import py_utils
from paxml import train_states


jax.distributed.initialize()

TrainState = train_states.TrainState
instantiate = base_hyperparams.instantiate
CheckpointType = checkpoints.CheckpointType
Checkpointer = checkpoints.Checkpointer
PaxCheckpointHandler = checkpoints.PaxCheckpointHandler


2023-08-04 11:19:09.228836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib


In [3]:
experiment_config = main.get_experiment('tasks.lm.params.c4.C4SpmdGpt37BRoPE')()
task_p = experiment_config.task()
jax_task = instantiate(task_p)

Registered experiment `paxml.tasks.lm.params.lm_cloud.LargeMlp`
Registered experiment `paxml.tasks.lm.params.lm_cloud.SmallMlp`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudTransformerAdam`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudTransformerAdamTest`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudTransformerAdamLimitSteps`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudSpmdTest`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudSpmd2B`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudSpmd2BLimitSteps`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudSpmd32B`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudSpmd64B`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudSpmd128B`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudSpmd256B`
Registered experiment `paxml.tasks.lm.params.lm_cloud.LmCloudSpmd512B`
Registered experiment `paxml.tasks.lm.params.lm_clou

In [4]:
options = checkpoint_managers.CheckpointManagerOptions(
      max_to_keep=10,
      save_interval_steps=20,
      cleanup_tmp_directories=True,
  )

checkpointer = Checkpointer(
          PaxCheckpointHandler(
              enforce_restore_shape_check=False,
              use_ocdbt=False,
          )
      )

job_log_dir = epath.Path('gs://llm_projects/log/lspdebug0804/checkpoints')
checkpoint_type = CheckpointType.GDA

checkpoint_manager = checkpoint_managers.OrbaxCheckpointManager(
      job_log_dir,
      checkpointer,
      train_input_checkpointer=False,
      options=options,
      checkpoint_type=checkpoint_type,
      tensorstore_use_ocdbt=False,
  )
# partitioner = partitioning.create_partitioner(
#         jax_task,
#         reshard_inputs=False,
#         auto_sharding_mode=None,
#     )

In [6]:
# # 输入只是为了拿到dtype和shape的

# NestedMap = py_utils.NestedMap

# seed = 0
# jax.random.PRNGKey(seed)

# my_sample_input = {}

# low, high = 0, 32000
# seq_length = 10
# # batch_size = experiment_config.PERCORE_BATCH_SIZE * 8
# batch_size = 1
# my_sample_input['ids'] = np.random.randint(low, high, (batch_size, seq_length)).astype(np.int32)
# my_sample_input['labels'] = my_sample_input['ids'].astype(np.int32)
# my_sample_input['weights'] = np.ones((batch_size, seq_length)).astype(np.float32)
# my_sample_input['paddings'] = my_sample_input['weights']
# my_sample_input['segment_ids'] = my_sample_input['weights'].astype(np.int32)
# my_sample_input['segment_pos'] = np.arange(seq_length).reshape(1, -1).repeat(batch_size, axis=0).astype(np.int32)
# my_sample_input['_seqio_provenance/shard_index'] = np.array([-1]).repeat(batch_size).astype(np.int32)
# my_sample_input['_seqio_provenance/num_shards'] = my_sample_input['_seqio_provenance/shard_index']
# my_sample_input['_seqio_provenance/index_within_shard'] = my_sample_input['_seqio_provenance/shard_index'].astype(np.int64)
# my_sample_input['eval_sample_weights'] = my_sample_input['_seqio_provenance/shard_index'].astype(np.float32)
# my_sample_input = NestedMap(my_sample_input)

# inputs_shape_dtype = jax.tree_map(
#         lambda x: jax.ShapeDtypeStruct(shape=x.shape, dtype=x.dtype),
#         my_sample_input,
#     )

# partitioner.setup(
#         jax_task,
#         None,
#         train_inputs_shape_dtype=inputs_shape_dtype, # None
#         train_input_pipeline=None,
#         job_log_dir=job_log_dir,
#     )

# train_state_metadata = partitioner.get_train_state_metadata()

# items = {'state': train_state_metadata.padded_global_shapes}
# restore_kwargs = {
#           'version': 1.1,
#           'specs': train_state_metadata.partition_specs, # shard
#           'mesh': partitioner.global_mesh, # mesh
#           'transforms': None, # None
#       }
# restore_kwargs = {'state': restore_kwargs}
# restored = checkpoint_manager._manager.restore(20, items=items, restore_kwargs=restore_kwargs)


In [18]:
gold_mngr_dir = epath.Path('gs://llm_base_models/orbax_async_test')
gold_item = {
                # 'opt_state': orbax.checkpoint.AsyncCheckpointer(orbax.checkpoint.PyTreeCheckpointHandler()),
                'params': orbax.checkpoint.AsyncCheckpointer(orbax.checkpoint.PyTreeCheckpointHandler()),
                # 'step': orbax.checkpoint.AsyncCheckpointer(orbax.checkpoint.ArrayCheckpointHandler()),
                }
gold_mngr = orbax.checkpoint.CheckpointManager(gold_mngr_dir, gold_item)
gold_w = gold_mngr.restore(gold_mngr.latest_step())

In [19]:
paxml_to_mesh_format = {
        ('params', 'lm', 'embedding_lookup', 'emb_var'): 'wte',
        ('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'ff_layer', 'ffn_layer1', 'linear', 'w'): 'w3',
        ('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'ff_layer', 'ffn_layer1_gate', 'linear', 'w'): 'w1',
        ('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'ff_layer', 'ffn_layer2', 'linear', 'w'): 'w2',
        ('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'self_attention', 'query', 'w'): 'wq',
        ('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'self_attention', 'key', 'w'): 'wk',
        ('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'self_attention', 'value', 'w'): 'wv',
        ('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'self_attention', 'post', 'w'): 'wo',
        ('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'layer_norm', 'scale'): 'attention_norm',
        ('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'ff_layer', 'layer_norm', 'scale'): 'ffn_norm',
        ('params', 'lm', 'final_ln', 'scale'): 'ln_f',
        ('params', 'lm', 'softmax', 'logits_ffn', 'linear', 'w'): 'lm_head',
    }

In [20]:
trans_result = {}

num_heads = experiment_config.NUM_HEADS
model_dims = experiment_config.MODEL_DIMS 
head_dim = model_dims // num_heads

for k, v in paxml_to_mesh_format.items():
    values = []
    for gold_key, glod_values in flatten_dict(gold_w['params']).items():
        if v in gold_key:
            if v in 'wqwkwvwo':
                glod_values = glod_values.reshape(model_dims, num_heads, head_dim)
            values.append([gold_key, glod_values])
    values = sorted(values, key=lambda x: x[0])
    if len(values) > 1:
        stack_values = np.stack(list(zip(*values))[1])
    else:
        stack_values = values[0][1]
    trans_result[k] = stack_values
opt_state_mv = jax.tree_map(lambda x: jnp.zeros_like(x), trans_result)

print(f'check....')

for k, v in trans_result.items():
    print(k, v.shape)


check....
('params', 'lm', 'embedding_lookup', 'emb_var') (32000, 4096)
('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'ff_layer', 'ffn_layer1', 'linear', 'w') (2, 4096, 11008)
('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'ff_layer', 'ffn_layer1_gate', 'linear', 'w') (2, 4096, 11008)
('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'ff_layer', 'ffn_layer2', 'linear', 'w') (2, 11008, 4096)
('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'self_attention', 'query', 'w') (2, 4096, 32, 128)
('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'self_attention', 'key', 'w') (2, 4096, 32, 128)
('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'self_attention', 'value', 'w') (2, 4096, 32, 128)
('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'self_attention', 'post', 'w') (2, 4096, 32, 128)
('params', 'lm', 'transformer', 'repeat', 'sub', 'x_layers_0', 'layer_norm', 'scale') (2, 4096)
('params', 

In [30]:
step = 200
n_layers = 2

temp_no_prefix, temp_other = {}, {}
for key_tuple, param in opt_state_mv.items():
    if 'repeat' in key_tuple:
        temp_no_prefix[key_tuple] = MaskedNode()
        temp_other[key_tuple] = param
    else:
        temp_no_prefix[key_tuple] = param
        temp_other[key_tuple] = MaskedNode()

temp_no_prefix = unflatten_dict(temp_no_prefix)
temp_other = unflatten_dict(temp_other)
    
no_prefix = {'count': jnp.array(step), 'm': temp_no_prefix, 'v': temp_no_prefix}
other = {'count': jnp.array([step] * n_layers), 'm': temp_other, 'v': temp_other}
trans_opt_states = {
                    'no_prefix': [{'count': jnp.array(step)}] * 2 + [no_prefix, {'count': jnp.array(step)}], 
                    f'p#{n_layers}#i-1': [{'count': jnp.array([step] * n_layers)}] * 2 + [other, {'count': jnp.array([step] * n_layers)}], 
}
trans_opt_states = [trans_opt_states]

new_trainstate = TrainState(step=jnp.array(step), 
                            mdl_vars=unflatten_dict(trans_result),
                            opt_states=trans_opt_states)

padded_global_shapes = jax.tree_map(lambda x: jax.ShapeDtypeStruct(shape=x.shape, dtype=x.dtype) 
                                    if hasattr(x, 'shape') else x , new_trainstate)
checkpoint_manager.save(step, new_trainstate, padded_global_shapes, train_input_pipeline=None, force=False)
print(f'Start load model to check whether saved model is True or False')
restored_model = checkpoint_manager._manager.restore(step, items=padded_global_shapes, restore_kwargs=restore_kwargs)
print(f'Load saved model finished. Congratulations!!!')

True